In [5]:
import pandas as pd
import os

# Function to load OULAD data from CSV files
# Using raw strings to handle backslashes in Windows file paths
# This function assumes the CSV files are located in the specified directory
# and that the directory structure matches the expected format.

def load_oulad_data(data_path):
    """Load all OULAD CSV files into pandas DataFrames"""

    # Core tables - using raw strings
    courses = pd.read_csv(rf"{data_path}\courses.csv")
    assessments = pd.read_csv(rf"{data_path}\assessments.csv")
    vle = pd.read_csv(rf"{data_path}\vle.csv")

    # Student data
    student_info = pd.read_csv(rf"{data_path}\studentInfo.csv")
    student_registration = pd.read_csv(rf"{data_path}\studentRegistration.csv")
    student_assessment = pd.read_csv(rf"{data_path}\studentAssessment.csv")
    student_vle = pd.read_csv(rf"{data_path}\studentVle.csv")

    return {
        'courses': courses,
        'assessments': assessments,
        'vle': vle,
        'student_info': student_info,
        'student_registration': student_registration,
        'student_assessment': student_assessment,
        'student_vle': student_vle
    }

# Load the data - use raw string for path
oulad_data = load_oulad_data(r"C:\Users\Ritam\Projects\XAIDashboard\dataset")

In [6]:
#data exploration
# Function to explore the main student information table
def explore_student_data(oulad_data):
    """Explore the main student information table"""

    student_info = oulad_data['student_info']

    print("=== STUDENT INFO EXPLORATION ===")
    print(f"Shape: {student_info.shape}")
    print(f"Columns: {list(student_info.columns)}")
    print("\n--- Sample Data ---")
    print(student_info.head(3))

    print("\n--- Target Variable Distribution ---")
    print(student_info['final_result'].value_counts())
    print(student_info['final_result'].value_counts(normalize=True).round(3))

    print("\n--- Demographic Breakdown ---")
    print("Gender:", student_info['gender'].value_counts().to_dict())
    print("Age bands:", student_info['age_band'].value_counts().to_dict())
    print("Disability:", student_info['disability'].value_counts().to_dict())

# Run exploration
explore_student_data(oulad_data)

=== STUDENT INFO EXPLORATION ===
Shape: (32593, 12)
Columns: ['code_module', 'code_presentation', 'id_student', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability', 'final_result']

--- Sample Data ---
  code_module code_presentation  id_student gender                region  \
0         AAA             2013J       11391      M   East Anglian Region   
1         AAA             2013J       28400      F              Scotland   
2         AAA             2013J       30268      F  North Western Region   

       highest_education imd_band age_band  num_of_prev_attempts  \
0       HE Qualification  90-100%     55<=                     0   
1       HE Qualification   20-30%    35-55                     0   
2  A Level or Equivalent   30-40%    35-55                     0   

   studied_credits disability final_result  
0              240          N         Pass  
1               60          N         Pass  
2               

In [7]:
#Check VLE Engagement Patterns
def explore_vle_data(oulad_data):
    """Explore VLE interaction patterns"""

    student_vle = oulad_data['student_vle']

    print("=== VLE INTERACTION EXPLORATION ===")
    print(f"Total interactions: {len(student_vle):,}")
    print(f"Unique students with VLE data: {student_vle['id_student'].nunique():,}")

    # Basic statistics
    print(f"Clicks per interaction - Mean: {student_vle['sum_click'].mean():.1f}, Max: {student_vle['sum_click'].max()}")

    # Activity patterns
    activity_types = oulad_data['vle']['activity_type'].value_counts().head(10)
    print("\n--- Top Activity Types ---")
    print(activity_types)

# Run VLE exploration
explore_vle_data(oulad_data)



=== VLE INTERACTION EXPLORATION ===
Total interactions: 10,655,280
Unique students with VLE data: 26,074
Clicks per interaction - Mean: 3.7, Max: 6977

--- Top Activity Types ---
activity_type
resource         2660
subpage          1055
oucontent         996
url               886
forumng           194
quiz              127
page              102
oucollaborate      82
questionnaire      61
ouwiki             49
Name: count, dtype: int64


In [8]:
#Assess Data Quality
def assess_data_quality(oulad_data):
    """Quick data quality assessment"""

    print("=== DATA QUALITY ASSESSMENT ===")

    for table_name, df in oulad_data.items():
        missing_pct = (df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
        print(f"{table_name}: {missing_pct:.1f}% missing values")

        if missing_pct > 0:
            missing_cols = df.isnull().sum()
            critical_missing = missing_cols[missing_cols > len(df) * 0.1]  # >10% missing
            if len(critical_missing) > 0:
                print(f"  ⚠️  High missingness in: {critical_missing.to_dict()}")

# Run quality assessment
assess_data_quality(oulad_data)


=== DATA QUALITY ASSESSMENT ===
courses: 0.0% missing values
assessments: 0.9% missing values
vle: 27.5% missing values
  ⚠️  High missingness in: {'week_from': 5243, 'week_to': 5243}
student_info: 0.3% missing values
student_registration: 13.8% missing values
  ⚠️  High missingness in: {'date_unregistration': 22521}
student_assessment: 0.0% missing values
student_vle: 0.0% missing values


In [9]:
# VLE Table: week_from and week_to (27.5% missing)
def handle_vle_missing(vle_data):
    """Handle missing week data in VLE table"""

    # Strategy 1: Create 'continuous' category for missing weeks
    vle_data['week_from'] = vle_data['week_from'].fillna(-1)  # -1 = continuous
    vle_data['week_to'] = vle_data['week_to'].fillna(-1)

    # Strategy 2: Create boolean flag for continuous activities
    vle_data['is_continuous_activity'] = vle_data['week_from'].isna()

    # Strategy 3: Fill with course-wide availability (0 to max_week)
    max_week = vle_data['week_to'].max()
    vle_data['week_from_filled'] = vle_data['week_from'].fillna(0)
    vle_data['week_to_filled'] = vle_data['week_to'].fillna(max_week)

    return vle_data

#Student Registration: date_unregistration (13.8% missing)
def handle_registration_missing(registration_data):
    """Handle missing unregistration dates"""

    # Strategy 1: Create binary completion indicator
    registration_data['completed_course'] = registration_data['date_unregistration'].isna()

    # Strategy 2: Fill with end-of-course date for completed students
    # (You can get course end dates from course table)
    registration_data['unregistration_status'] = registration_data['date_unregistration'].apply(
        lambda x: 'completed' if pd.isna(x) else 'withdrew'
    )

    # Strategy 3: Calculate retention days (for non-missing values)
    registration_data['retention_days'] = (
        registration_data['date_unregistration'] - registration_data['date_registration']
    ).dt.days

    return registration_data


In [10]:
#Handle all missing values comprehensively
def comprehensive_missing_value_handling(oulad_data):
    """Complete strategy for handling all missing values"""

    print("=== HANDLING MISSING VALUES ===")

    # 1. VLE table - handle week missingness
    print("Processing VLE missing values...")
    vle = oulad_data['vle'].copy()

    # Create flags for continuous activities
    vle['is_continuous'] = vle['week_from'].isna()

    # Fill with meaningful values
    vle['week_from'] = vle['week_from'].fillna(0)  # Start of course
    vle['week_to'] = vle['week_to'].fillna(vle['week_to'].max())  # End of course

    oulad_data['vle'] = vle
    print(f"✅ VLE: Added continuous activity flags")

    # 2. Student registration - handle unregistration missingness
    print("Processing registration missing values...")
    registration = oulad_data['student_registration'].copy()

    # Create completion indicators
    registration['completed_course'] = registration['date_unregistration'].isna()
    registration['withdrawal_status'] = registration['date_unregistration'].apply(
        lambda x: 'completed' if pd.isna(x) else 'withdrew_early'
    )

    oulad_data['student_registration'] = registration
    print(f"✅ Registration: Added completion indicators")

    # 3. Handle minor missing values in other tables
    print("Processing minor missing values...")

    # Student info - forward fill demographic data (assumed stable)
    student_info = oulad_data['student_info'].copy()
    demographic_cols = ['gender', 'region', 'highest_education', 'disability']
    for col in demographic_cols:
        if col in student_info.columns and student_info[col].isna().any():
            # Fill with mode (most common value)
            mode_value = student_info[col].mode()[0]
            student_info[col] = student_info[col].fillna(mode_value)
            print(f"   Filled {col} with mode: {mode_value}")

    oulad_data['student_info'] = student_info

    # 4. Verify no critical missing values remain
    print("\n=== POST-PROCESSING VERIFICATION ===")
    for table_name, df in oulad_data.items():
        critical_missing = df.isnull().sum().sum()
        if critical_missing > 0:
            print(f"⚠️  {table_name}: {critical_missing} missing values remain")
            missing_cols = df.isnull().sum()
            print(f"   Columns: {missing_cols[missing_cols > 0].to_dict()}")
        else:
            print(f"✅ {table_name}: No missing values")

    return oulad_data

# Apply the comprehensive handling
oulad_data_clean = comprehensive_missing_value_handling(oulad_data)


=== HANDLING MISSING VALUES ===
Processing VLE missing values...
✅ VLE: Added continuous activity flags
Processing registration missing values...
✅ Registration: Added completion indicators
Processing minor missing values...

=== POST-PROCESSING VERIFICATION ===
✅ courses: No missing values
⚠️  assessments: 11 missing values remain
   Columns: {'date': 11}
✅ vle: No missing values
⚠️  student_info: 1111 missing values remain
   Columns: {'imd_band': 1111}
⚠️  student_registration: 22566 missing values remain
   Columns: {'date_registration': 45, 'date_unregistration': 22521}
⚠️  student_assessment: 173 missing values remain
   Columns: {'score': 173}
✅ student_vle: No missing values


In [12]:
#4. Feature Engineering from Missing Value Patterns
def create_missing_value_features(oulad_data_clean):
    """Create predictive features from missing value patterns"""

    # From VLE data
    vle_features = oulad_data_clean['vle'].groupby(['code_module', 'code_presentation']).agg({
        'is_continuous': 'sum',  # Count of continuous activities per course
    }).reset_index()
    vle_features.rename(columns={'is_continuous': 'continuous_activities_count'}, inplace=True)

    # From registration data
    registration_features = oulad_data_clean['student_registration'][
        ['code_module', 'code_presentation', 'id_student', 'completed_course', 'withdrawal_status']
    ]

    print("✅ Created predictive features from missing value patterns")

    return vle_features, registration_features

def verify_missing_handling(original_data, clean_data):
    """Verify missing value handling worked correctly"""

    print("=== MISSING VALUE HANDLING VERIFICATION ===")

    for table_name in original_data.keys():
        original_missing = original_data[table_name].isnull().sum().sum()
        clean_missing = clean_data[table_name].isnull().sum().sum()

        print(f"{table_name}:")
        print(f"  Before: {original_missing} missing values")
        print(f"  After:  {clean_missing} missing values")
        print(f"  Reduction: {original_missing - clean_missing}")

        # Check if any new features were created
        original_cols = set(original_data[table_name].columns)
        clean_cols = set(clean_data[table_name].columns)
        new_features = clean_cols - original_cols
        if new_features:
            print(f"  ✅ New features: {new_features}")
        print()

# Run verification
verify_missing_handling(oulad_data, oulad_data_clean)



=== MISSING VALUE HANDLING VERIFICATION ===
courses:
  Before: 0 missing values
  After:  0 missing values
  Reduction: 0

assessments:
  Before: 11 missing values
  After:  11 missing values
  Reduction: 0

vle:
  Before: 0 missing values
  After:  0 missing values
  Reduction: 0

student_info:
  Before: 1111 missing values
  After:  1111 missing values
  Reduction: 0

student_registration:
  Before: 22566 missing values
  After:  22566 missing values
  Reduction: 0

student_assessment:
  Before: 173 missing values
  After:  173 missing values
  Reduction: 0

student_vle:
  Before: 0 missing values
  After:  0 missing values
  Reduction: 0

